# Transfer Learning

**Transfer Learning:**

A research problem in machine learning that focuses on storing knowledge gained while solving one problem and applying it to a different but related problem. 

Can leverage an existing neural network architecture proven to work on problems similar to our own

Can leverage a working network architecture which has already learned patterns on similar data to our own (often results in great results with less data)

**Transfer learning use cases:**
* Computer Vision
  - [Imagenet](https://www.image-net.org/): image database organized according to the WordNet hierarchy in which each node of the hierarchy is depicted by hundreds and thousands of images.
  - Currently the best architecture is called EfficientNet
* Natural Language Processing:
  - A subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data. 

## Feature Extraction

In [1]:
import tensorflow as tf
physical_device = tf.config.experimental.list_physical_devices('GPU')
print(f'Device found : {physical_device}') 

Device found : []


In [2]:
# Get Data (10% of 10 food classes from Food101)
import zipfile

# Download data
!wget -nc -P ../Downloads/ https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip
zip_ref = zipfile.ZipFile('../Downloads/10_food_classes_10_percent.zip')
zip_ref.extractall(path='../Downloads/')
zip_ref.close()


File ‘../Downloads/10_food_classes_10_percent.zip’ already there; not retrieving.



In [3]:
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk('../Downloads/10_food_classes_10_percent'):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '../Downloads/10_food_classes_10_percent'.
There are 10 directories and 0 images in '../Downloads/10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '../Downloads/10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '../Downloads/10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '../Downloads/10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '../Downloads/10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '../Downloads/10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '../Downloads/10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '../Downloads/10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '../Downloads/10_food_classes_10_percent/test/pizza'.
There are 0 directories and

### Create data loaders (preparing the data using `ImageDataGenerator`)

In [4]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = '../Downloads/10_food_classes_10_percent/train/'
test_dir = '../Downloads/10_food_classes_10_percent/test/'

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print('Training images:')
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode='categorical')

print('Testing images:')
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode='categorical')

Training images:
Found 750 images belonging to 10 classes.
Testing images:
Found 2500 images belonging to 10 classes.


### Setting up callbacks

callbacks are extra functionality you can add to your models to be performed during or after training.  Some of the most popular callbacks:
* Tracking experiments with the `TensorBoard()` callback
  - Log the performance of multiple models and then view and compare these models in a visual way on TensorBoard (a dashboard for inspecting neural network parameters).  Helpful to compare the results of different models on your data.
* Model checkpoint with the `ModelCheckpoint()` callback
  - Save your model as it trains so you can stop training if needed and come back to continue where you left off.  Helpful if training takes a long time and can't be done in one sitting.
* Stopping a model from training (before it trains too long and overfits) with the `EarlyStopping()` callback
  - Leave your model training for an arbitrary amount of time and have it stop training automaticaly when it ceases to improve.  Helpful when you've got a large dataset and don't know how long training will take.

Can be accessed via `tf.keras.callbacks`

In [5]:
# Create tensorboard callback (functionized because need to create a new one for each model)
import datetime
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

### Creating models using TensorFlow Hub

TensorFlow Hub is a repo of trained ml models

we can access pretrained models on: https://tfhub.dev/

**ResNet**
- Deep Residual Learning for Image Recognition.


In [6]:
# Compare the following two models
resnet_url = 'https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5'
efficientnet_url = 'https://tfhub.dev/google/efficientnet/b0/feature_vector/1'



In [7]:
# Import dependencies
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [8]:
# make a create_model() function to create a model from a URL
def create_model(model_url, num_classes=10):
    """
    Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.

    Args:
        model_url (str): A TensorFlow Hub feature extraction URL.
        num_classes (int): Number of output neurons in the output layer, 
            should be equal to number of target classes, default 10.as_integer_ratio
    Returns:
        An uncompiled Keras Sequential model with model_url as feature extractor
        layer and Dense output layer with num_classes output neurons.
    """
    # Create the model
    model = tf.keras.Sequential()
    # Create Sequential layer from existing url
    feature_extractor_layer = hub.KerasLayer(model_url, 
            trainable=False,
            name="feature_extraction_layer",
            input_shape=IMAGE_SHAPE + (3,)
        ) # freeze the already learned patterns
    # Create layers
    output_layer = layers.Dense(num_classes, activation='softmax', name='output_layer')
    
    # Add layers to the model
    model.add(feature_extractor_layer)
    model.add(output_layer)

    return model


### Creating and testing ResNet TensorFlow Hub Feature Extraction model

In [9]:
# Create ResNet model
resnet_model = create_model(resnet_url, num_classes=train_data_10_percent.num_classes)
resnet_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
feature_extraction_layer (Ke (None, 2048)              23564800  
_________________________________________________________________
output_layer (Dense)         (None, 10)                20490     
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


In [10]:
# Compile our resnet model
resnet_model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

In [11]:
# Fit the model
resnet_history = resnet_model.fit(train_data_10_percent,
                                  epochs=5,
                                  steps_per_epoch=len(train_data_10_percent),
                                  validation_data=test_data,
                                  validation_steps=len(test_data),
                                  # Add TensorBoard callback to model (callbacks parameter takes a list)
                                  callbacks=[create_tensorboard_callback(dir_name="../tensorflow_hub", # save experiment logs here
                                                                         experiment_name="resnet50V2")]) # name of log files

Saving TensorBoard log files to: ../tensorflow_hub/resnet50V2/20220803-135401
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'list' object has no attribute 'name'


Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'list' object has no attribute 'name'


Epoch 1/5
24/24 [==============================] - 15s 646ms/step - loss: 1.7885 - acc: 0.4267 - val_loss: 1.0938 - val_acc: 0.6692
Epoch 2/5
24/24 [==============================] - 9s 360ms/step - loss: 0.8545 - acc: 0.7573 - val_loss: 0.8027 - val_acc: 0.7488
Epoch 3/5
24/24 [==============================] - 9s 366ms/step - loss: 0.5872 - acc: 0.8387 - val_loss: 0.7261 - val_acc: 0.7624
Epoch 4/5
24/24 [==============================] - 9s 361ms/step - loss: 0.4620 - acc: 0.8827 - val_loss: 0.6819 - val_acc: 0.7752
Epoch 5/5
24/24 [==============================] - 9s 358ms/step - loss: 0.3719 - acc: 0.9067 - val_loss: 0.6563 - val_acc: 0.7832
